# Forward and Reverse mode gradients in TFF

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [10]:
import tensorflow as tf
import functools
import tf_quant_finance.math.gradient as tff_gradient
import tf_quant_finance.math.optimizer as tff_optimizer
import numpy as np

Consider a simple function in two variables $x$ and $y$ but in three dimensions such that the function along each axes is [$x^2$, $y^2$, $x y$]

$
\begin{align}
& f = [f_1, f_2, f_3] \\
& where \\
\end{align}
$

$
\begin{align}
f_1 &= x^2 \\
f_2 &= y^2 \\
f_3 &= x + y \\
\end{align}
$

In [18]:
def func_grad(x):
    func = tf.stack([x[0]**2, x[1]**2, x[0]*x[1]])
    grad = tf.stack([2.0*x[0]+x[1], 2.0*x[1]+x[0]])
    return func, grad

def func(x):
    func = tf.stack([x[0]**2, x[1]**2, x[0]*x[1]])
    return func

start = tf.constant([1,1], dtype=tf.float64)


# Reverse mode

$
\begin{align}
\frac {\partial f}{\partial x} &= u_1 \frac{\partial f_1}{\partial x} + u_2 \frac{\partial f_2}{\partial x} + u_3 \frac{\partial f_3}{\partial x} \\ \\
&= u_1 2 x + u_3 y\\ \\
\frac {\partial f}{\partial y} &= u_1 \frac{\partial f_1}{\partial y} + u_2 \frac{\partial f_2}{\partial y} + u_3 \frac{\partial f_3}{\partial y} \\ \\
&= u_1 2 y + u_3 x
\end{align}
$

In Tensorflow, [$u_1$, $u_2$, $u_3$] is normally set to [1, 1, 1]

Setting [$x$, $y$] to [1,1]

Reverse mode returns the gradients summed up by components

In [25]:
func_grad(start)[1]

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([3., 3.])>

# Forward mode

$
\begin{align}
{\partial f_1} &= w_1 \frac{\partial f_1}{\partial x} + w_2 \frac{\partial f_1}{\partial y} \\ \\
&= w_1 2 x \\ \\
{\partial f_2} &= w_1 \frac{\partial f_2}{\partial x} + w_2 \frac{\partial f_2}{\partial y} \\ \\
&= w_2 2 y \\ \\
{\partial f_3} &= w_1 \frac{\partial f_3}{\partial x} + w_2 \frac{\partial f_3}{\partial y} \\ \\
&= w_1 x + w_2 y \\ \\
\end{align}
$

In Tensorflow, [$w_1$, $w_2$] is normally set to [1, 1]

Setting [$x$, $y$] to [1,1]

Forward mode returns the gradients by components

In [26]:
tff_gradient.fwd_gradient(func, start)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 2., 2.])>